In [1]:
%matplotlib ipympl
from matplotlib import pylab as plt
import matplotlib
import molpx
import mdtraj as md

import numpy as np
import pandas as pd

import pytraj as pt
import pyemma

#import matplotlib 
#import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys
import glob

_ColormakerRegistry()

## Load reference structures

In [2]:
rf0=pt.load('./parm_rst/closed6_dry_rot0.rst7', './parm_rst/closed6_dry_rot0.parm7')
rf1=pt.load('./parm_rst/closed6_dry_rot1.rst7', './parm_rst/closed6_dry_rot1.parm7')
rf2=pt.load('./parm_rst/closed6_dry_rot2.rst7', './parm_rst/closed6_dry_rot2.parm7')
rf3=pt.load('./parm_rst/closed6_dry_rot3.rst7', './parm_rst/closed6_dry_rot3.parm7')
rf4=pt.load('./parm_rst/closed6_dry_rot4.rst7', './parm_rst/closed6_dry_rot4.parm7')
rf5=pt.load('./parm_rst/closed6_dry_rot5.rst7', './parm_rst/closed6_dry_rot5.parm7')

rfopen=pt.load('./parm_rst/open10_dry.rst7', './parm_rst/open10_dry.parm7')

## Free energy surface plot

List the trajectories

In [3]:
indir = './traj_dry_open_closed_adapt_all'
topfile = './parm_rst/open10_dry.parm7'
from glob import glob
traj_list = sorted(glob(indir+'/*.nc'))
traj_list

['./traj_dry_open_closed_adapt_all/adapt1_dry_1.nc',
 './traj_dry_open_closed_adapt_all/adapt1_dry_10.nc',
 './traj_dry_open_closed_adapt_all/adapt1_dry_100.nc',
 './traj_dry_open_closed_adapt_all/adapt1_dry_101.nc',
 './traj_dry_open_closed_adapt_all/adapt1_dry_102.nc',
 './traj_dry_open_closed_adapt_all/adapt1_dry_105.nc',
 './traj_dry_open_closed_adapt_all/adapt1_dry_11.nc',
 './traj_dry_open_closed_adapt_all/adapt1_dry_110.nc',
 './traj_dry_open_closed_adapt_all/adapt1_dry_114.nc',
 './traj_dry_open_closed_adapt_all/adapt1_dry_115.nc',
 './traj_dry_open_closed_adapt_all/adapt1_dry_117.nc',
 './traj_dry_open_closed_adapt_all/adapt1_dry_118.nc',
 './traj_dry_open_closed_adapt_all/adapt1_dry_125.nc',
 './traj_dry_open_closed_adapt_all/adapt1_dry_129.nc',
 './traj_dry_open_closed_adapt_all/adapt1_dry_13.nc',
 './traj_dry_open_closed_adapt_all/adapt1_dry_132.nc',
 './traj_dry_open_closed_adapt_all/adapt1_dry_133.nc',
 './traj_dry_open_closed_adapt_all/adapt1_dry_134.nc',
 './traj_dry_op

Calculate the RMSD of each trajectory frames with one of teh reference structure. 
6 reference structures are used for the closed form in order to account to chain symmetry. Only the minimum RMSD value for each frame with respect to the closed crystal structures is kept

In [4]:
results_open=[]
results_closed=[]
for t in traj_list:
    traj=pt.load(t, topfile)
    rmsd0=pt.rmsd(traj,ref=rf0, mask="@CA,N,C,O")
    rmsd1=pt.rmsd(traj,ref=rf1, mask="@CA,N,C,O")
    rmsd2=pt.rmsd(traj,ref=rf2, mask="@CA,N,C,O")
    rmsd3=pt.rmsd(traj,ref=rf3, mask="@CA,N,C,O")
    rmsd4=pt.rmsd(traj,ref=rf4, mask="@CA,N,C,O")
    rmsd5=pt.rmsd(traj,ref=rf5, mask="@CA,N,C,O")
    
    rmsd_closed=np.nanmin(np.array([rmsd0,rmsd1,rmsd2,rmsd3,rmsd4,rmsd5]), axis=0)
    results_closed.append(rmsd_closed)
    
    rmsd_open=pt.rmsd(traj,ref=rfopen, mask="@CA,N,C,O")
    results_open.append(rmsd_open)

Create a Y matrix similar to that optained with PyEMMA after TICA analysis

In [5]:
Y=[]
for i in range(0,len(results_open)):  
    Z=[]
    for j in range(0,len(results_open[i])):
        a=np.array([results_closed[i][j],results_open[i][j]])
        Z.append(a)
    Z=np.array(Z)
    Y.append(Z)
print(len(Y))
print(len(Y[0]))
print(len(Y[0][0]))

146
1000
2


Plot interactive FES 

In [6]:
mpx_wdg_box = molpx.visualize.FES(traj_list, topfile, Y, nbins=200, n_sample= 1000,
                                  proj_labels=[r'RMSD to closed crystal structure ($\AA$)',
                                               r'RMSD to open crystal structure ($\AA$)'])
mpx_wdg_box.linked_ngl_wdgs[0].camera = 'orthographic'
mpx_wdg_box

17-12-19 14:20:03 pyemma.coordinates.clustering.regspace.RegularSpaceClustering[0] WARNING  Maximum number of cluster centers reached. Consider increasing max_centers or choose a larger minimum distance, dmin.
17-12-19 14:20:03 pyemma.coordinates.clustering.regspace.RegularSpaceClustering[0] INFO     Presumably finished estimation. Message: Used data for centers: 0.97%


/home/el14718/Software/anaconda3_2019/lib/python3.7/site-packages/pyemma/coordinates/clustering/regspace.py:153: UserWarning: Maximum number of cluster centers reached. Consider increasing max_centers or choose a larger minimum distance, dmin.
  warnings.warn(msg)


17-12-19 14:20:09 pyemma.coordinates.clustering.regspace.RegularSpaceClustering[2] WARNING  Maximum number of cluster centers reached. Consider increasing max_centers or choose a larger minimum distance, dmin.
17-12-19 14:20:09 pyemma.coordinates.clustering.regspace.RegularSpaceClustering[2] INFO     Presumably finished estimation. Message: Used data for centers: 6.12%


/home/el14718/Software/anaconda3_2019/lib/python3.7/site-packages/pyemma/coordinates/clustering/regspace.py:153: UserWarning: Maximum number of cluster centers reached. Consider increasing max_centers or choose a larger minimum distance, dmin.
  warnings.warn(msg)


MolPXHBox(children=(NGLWidget(max_frame=862), FigureCanvasNbAgg()))

In [43]:
mpx_wdg_box.linked_ngl_wdgs[0].remove_line()
mpx_wdg_box.linked_ngl_wdgs[0].remove_licorice()
mpx_wdg_box.linked_ngl_wdgs[0].remove_surface()

In [42]:
mpx_wdg_box.linked_ngl_wdgs[0].add_line(selection="(not hydrogen) and (not backbone)")

In [ ]:
mpx_wdg_box.linked_ngl_wdgs[0].add_surface(selection="all", surfacetype="ms",probeRadius=0.8 )

K-mean clustering

In [ ]:
n_clusters = 14
clustering = pyemma.coordinates.cluster_kmeans(Y,k=n_clusters, max_iter=1000, fixed_seed=True,
                                               n_jobs=1, init_strategy='uniform')

In [ ]:
def plot_labels(ax=None):
    #if ax is None:
        #ax = gca()
    for i in range(0,len(clustering.clustercenters)):
        plt.text(clustering.clustercenters[i][0]+0.04, clustering.clustercenters[i][1]+0.04, 
                 i, fontsize=10, color='white')


In [ ]:
pyemma.plots.plot_free_energy(np.vstack(Y)[:, 0], np.vstack(Y)[:, 1])
cc_x = clustering.clustercenters[:, 0]
cc_y = clustering.clustercenters[:, 1]
plt.plot(cc_x, cc_y, linewidth=0, marker='o', markersize=5, color='white')
plot_labels()

Extract representative structures

In [ ]:
clkmeans = pyemma.coordinates.cluster_kmeans([iY[:,:2] for iY in Y], 10)

In [ ]:
data_sample, geoms = molpx.generate.sample(traj_list, topfile, clkmeans,n_geom_samples=50)

## Free energy surface for IPA containing simulations

In [7]:
indir = './traj_IPA_dry'
topfile = './parm_rst/open10_dry.parm7'
from glob import glob
traj_list2 = sorted(glob(indir+'/*.nc'))
traj_list2

['./traj_IPA_dry/open_ipa_dry_1.nc',
 './traj_IPA_dry/open_ipa_dry_2.nc',
 './traj_IPA_dry/open_ipa_dry_3.nc',
 './traj_IPA_dry/open_ipa_dry_4.nc',
 './traj_IPA_dry/open_ipa_dry_5.nc']

In [8]:
results_open_ipa=[]
results_closed_ipa=[]
for t in traj_list2:
    traj=pt.load(t, topfile)
    rmsd0=pt.rmsd(traj,ref=rf0, mask="@CA,N,C,O")
    rmsd1=pt.rmsd(traj,ref=rf1, mask="@CA,N,C,O")
    rmsd2=pt.rmsd(traj,ref=rf2, mask="@CA,N,C,O")
    rmsd3=pt.rmsd(traj,ref=rf3, mask="@CA,N,C,O")
    rmsd4=pt.rmsd(traj,ref=rf4, mask="@CA,N,C,O")
    rmsd5=pt.rmsd(traj,ref=rf5, mask="@CA,N,C,O")
    
    rmsd_closed_ipa=np.nanmin(np.array([rmsd0,rmsd1,rmsd2,rmsd3,rmsd4,rmsd5]), axis=0)
    results_closed_ipa.append(rmsd_closed_ipa)
    
    rmsd_open_ipa=pt.rmsd(traj,ref=rfopen, mask="@CA,N,C,O")
    results_open_ipa.append(rmsd_open_ipa)

In [9]:
Y_ipa=[]
for i in range(0,len(results_open_ipa)):  
    Z_ipa=[]
    for j in range(0,len(results_open_ipa[i])):
        a_ipa=np.array([results_closed_ipa[i][j],results_open_ipa[i][j]])
        Z_ipa.append(a_ipa)
    Z_ipa=np.array(Z_ipa)
    Y_ipa.append(Z_ipa)

In [10]:
mpx_wdg_box_ipa = molpx.visualize.FES(traj_list2, topfile, Y_ipa, nbins=200, n_sample= 500,
                                  proj_labels=[r'RMSD to closed crystal structure ($\AA$)',
                                               r'RMSD to open crystal structure ($\AA$)'])
mpx_wdg_box_ipa.linked_ngl_wdgs[0].camera = 'orthographic'
mpx_wdg_box_ipa

17-12-19 14:32:15 pyemma.coordinates.clustering.regspace.RegularSpaceClustering[23] WARNING  Maximum number of cluster centers reached. Consider increasing max_centers or choose a larger minimum distance, dmin.
17-12-19 14:32:15 pyemma.coordinates.clustering.regspace.RegularSpaceClustering[23] INFO     Presumably finished estimation. Message: Used data for centers: 40.00%


/home/el14718/Software/anaconda3_2019/lib/python3.7/site-packages/pyemma/coordinates/clustering/regspace.py:153: UserWarning: Maximum number of cluster centers reached. Consider increasing max_centers or choose a larger minimum distance, dmin.
  warnings.warn(msg)


MolPXHBox(children=(NGLWidget(max_frame=481), FigureCanvasNbAgg()))

In [41]:
mpx_wdg_box_ipa.linked_ngl_wdgs[0].remove_line()
mpx_wdg_box_ipa.linked_ngl_wdgs[0].remove_licorice()
mpx_wdg_box_ipa.linked_ngl_wdgs[0].remove_surface()

In [35]:
mpx_wdg_box_ipa.linked_ngl_wdgs[0].add_surface(selection="all", surfacetype="ms",probeRadius=0.8 )

In [36]:
mpx_wdg_box_ipa.linked_ngl_wdgs[0].add_line(selection="(not hydrogen) and (not backbone)")

In [ ]:
rmsd_open=np.concatenate(results_open, axis=0)
rmsd_closed=np.concatenate(results_closed, axis=0)

In [ ]:
plt.plot(rmsd_closed, rmsd_open, marker=".", ms=0.5, ls='None') 

In [ ]:
pyemma.plots.plot_free_energy(rmsd_closed, rmsd_open, nbins=200)

In [ ]:
n_clusters = 8
clustering = pyemma.coordinates.cluster_kmeans(Y,k=n_clusters, max_iter=1000, n_jobs=1)

In [ ]:
clustering.clustercenters

In [ ]:
def plot_labels(ax=None):
    for i in range(0,len(clustering.clustercenters)):
        plt.text(clustering.clustercenters[i][0]+0.06, clustering.clustercenters[i][1]+0.07, 
                 i, fontsize=10, color='black')

In [ ]:
pyemma.plots.plot_free_energy(np.vstack(Y)[:, 0], np.vstack(Y)[:, 1], nbins=200)
cc_x = clustering.clustercenters[:, 0]
cc_y = clustering.clustercenters[:, 1]
plt.plot(cc_x, cc_y, linewidth=0, marker='o', markersize=5, color='white')
plot_labels()

In [ ]:
import molpx
import mdtraj as md

In [ ]:
indir = './Trajectories_all'
topfile =  indir+'/open10_dry.parm7'
from glob import glob
traj_list = sorted(glob(indir+'/*.nc'))
traj_list

In [ ]:
mpx_wdg_box = molpx.visualize.FES(traj_list,
                                 #MD_trajfiles,
                                 topfile,
                                 Y,
                                 #Y,
                                 nbins=50,
                                 #proj_idxs=[1,2],
                                 #proj_labels='RMSD',
                                 #n_overlays=5,
                                )
mpx_wdg_box